<a href="https://colab.research.google.com/github/pattichis/AdvancedPython/blob/main/Decorators_and_Numba.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Speed up Python execution 10X to 50X with one line of code!
Click on Runtime and select different environments to see different speedups.

The method is exteremely effective on GPUs.


Here are the speedups on the A100:
```
Results are approximately equal: True
Plain Python Time: 0.9012 seconds
Numba Jitted Time: 0.0220 seconds

Numba provided a speedup of approximately 40.88 times!
```

The standard Python libraries work well:
```
import math
from numba import njit
import numpy as np # Import NumPy
```
Then all you need to do is:
```
@njit
def function_name(...):
    ...
```




### Decorators and Numba

 **Decorators** are a syntactic sugar for higher-order functions that take another function as an argument, and return a new function (or a modified version of the original). This allows you to 'wrap' functions with additional functionality without directly modifying their code.

Let's create a decorator that logs when a function is called and with what arguments, and also logs its return value.

In [ ]:
def log_function_call(func):
    """A decorator that logs function calls, arguments, and return values."""
    def wrapper(*args, **kwargs):
        print(f"\n--- Calling function: '{func.__name__}' ---")
        print(f"Arguments: {args}, Keyword Arguments: {kwargs}")

        # Call the original function
        result = func(*args, **kwargs)

        print(f"Function '{func.__name__}' returned: {result}")
        print(f"--- End of call for '{func.__name__}' ---")
        return result
    return wrapper


# Now, let's define some functions and apply our decorator to them.
@log_function_call
def add(a, b):
    """Adds two numbers."""
    return a + b

@log_function_call
def multiply(x, y, z=1):
    """Multiplies three numbers, with a default for z."""
    return x * y * z

@log_function_call
def greet(name):
    """Greets a person."""
    return f"Hello, {name}!"


print("Calling decorated functions:")

# Call the decorated functions
add_result = add(5, 3)
print(f"Result of add(5, 3): {add_result}")

multiply_result = multiply(2, 4, z=10)
print(f"Result of multiply(2, 4, z=10): {multiply_result}")

greet_message = greet("Alice")
print(f"Result of greet('Alice'): {greet_message}")

Calling decorated functions:

--- Calling function: 'add' ---
Arguments: (5, 3), Keyword Arguments: {}
Function 'add' returned: 8
--- End of call for 'add' ---
Result of add(5, 3): 8

--- Calling function: 'multiply' ---
Arguments: (2, 4), Keyword Arguments: {'z': 10}
Function 'multiply' returned: 80
--- End of call for 'multiply' ---
Result of multiply(2, 4, z=10): 80

--- Calling function: 'greet' ---
Arguments: ('Alice',), Keyword Arguments: {}
Function 'greet' returned: Hello, Alice!
--- End of call for 'greet' ---
Result of greet('Alice'): Hello, Alice!


### Explanation

1.  **`log_function_call(func)`**: This is our decorator function. It takes another function (`func`) as its argument.
2.  **`wrapper(*args, **kwargs)`**: Inside the decorator, we define an inner function called `wrapper`. This `wrapper` function is what will actually replace the original `func`. It uses `*args` and `**kwargs` to accept any number of positional and keyword arguments, making it flexible enough to wrap any function.
3.  **Logging Logic**: Inside `wrapper`, we add our logging code *before* and *after* calling the original function `func(*args, **kwargs)`.
4.  **`return wrapper`**: The decorator returns this `wrapper` function. When you use `@log_function_call` above a function definition, it's equivalent to `add = log_function_call(add)`. So, the `add` variable now points to our `wrapper` function.
5.  **`@log_function_call` Syntax**: This is syntactic sugar. When you write:
    ```python
    @log_function_call
    def add(a, b):
        # ...
    ```
    Python automatically translates it to:
    ```python
    def add(a, b):
        # ...
    add = log_function_call(add)
    ```

This example showcases metaprogramming because the decorator (`log_function_call`) takes a function definition and returns a *new* function definition that includes extra logging capabilities, all done dynamically at definition time without altering the original function's core logic.

### Applying Numba to a Python Function

Numba is a Just-In-Time (JIT) compiler for Python that translates Python functions to optimized machine code at runtime. It's especially effective for numerical algorithms that involve loops, NumPy arrays, and mathematical operations.

Let's apply Numba to our `sum_of_sqrts` function to see its performance impact.

In [ ]:
import time
import math
from numba import njit
import numpy as np # Import NumPy

# Generate a large list of numbers (same as before)
n_elements_numba = 10**7 # 10 million elements
# Convert to a NumPy array for Numba optimization
data_numba = np.array(list(range(1, n_elements_numba + 1)), dtype=np.float64) # Use float64 for sqrt compatibility

#### 1. Plain Python Function (Imperative Approach)

In [ ]:
def sum_of_sqrts_plain_python(numbers):
    total = 0.0
    for num in numbers:
        total += math.sqrt(num)
    return total

# Warmup run (already done in previous cells, but for self-containment)
_ = sum_of_sqrts_plain_python(data_numba)

start_time_plain = time.time()
plain_result = sum_of_sqrts_plain_python(data_numba)
end_time_plain = time.time()
plain_time = end_time_plain - start_time_plain

print(f"Plain Python Result: {plain_result:.4f}")
print(f"Plain Python Time: {plain_time:.4f} seconds")

Plain Python Result: 21081852648.7170
Plain Python Time: 0.8812 seconds


#### 2. Numba Jitted Function

In [ ]:
@njit
def sum_of_sqrts_numba(numbers):
    total = 0.0
    for num in numbers:
        total += np.sqrt(num) # Use np.sqrt for better Numba integration with NumPy arrays
    return total

# Numba functions have a 'compilation' overhead on the first call.
# We run a warmup call to ensure the timing reflects the compiled version.
_ = sum_of_sqrts_numba(data_numba[:100]) # Use a small slice for warmup

start_time_numba = time.time()
numba_result = sum_of_sqrts_numba(data_numba)
end_time_numba = time.time()
numba_time = end_time_numba - start_time_numba

print(f"Numba Jitted Result: {numba_result:.4f}")
print(f"Numba Jitted Time: {numba_time:.4f} seconds")

Numba Jitted Result: 21081852648.7170
Numba Jitted Time: 0.0218 seconds


In [ ]:
@njit
def sum_of_sqrts_numba(numbers):
    total = 0.0
    for num in numbers:
        total += math.sqrt(num)
    return total

# Numba functions have a 'compilation' overhead on the first call.
# We run a warmup call to ensure the timing reflects the compiled version.
_ = sum_of_sqrts_numba(data_numba[:100]) # Use a small slice for warmup

start_time_numba = time.time()
numba_result = sum_of_sqrts_numba(data_numba)
end_time_numba = time.time()
numba_time = end_time_numba - start_time_numba

print(f"Numba Jitted Result: {numba_result:.4f}")
print(f"Numba Jitted Time: {numba_time:.4f} seconds")

Numba Jitted Result: 21081852648.7170
Numba Jitted Time: 0.0218 seconds


In [ ]:
print(f"\nResults are approximately equal: {abs(plain_result - numba_result) < 1e-9}")
print(f"Plain Python Time: {plain_time:.4f} seconds")
print(f"Numba Jitted Time: {numba_time:.4f} seconds")

if numba_time < plain_time:
    speedup_factor = plain_time / numba_time
    print(f"\nNumba provided a speedup of approximately {speedup_factor:.2f} times!")
else:
    print("\nNumba did not provide a speedup in this instance, or the times were similar.")


Results are approximately equal: True
Plain Python Time: 0.8812 seconds
Numba Jitted Time: 0.0218 seconds

Numba provided a speedup of approximately 40.41 times!


#### 3. Comparison

In [ ]:
print(f"\nResults are approximately equal: {abs(plain_result - numba_result) < 1e-9}")
print(f"Plain Python Time: {plain_time:.4f} seconds")
print(f"Numba Jitted Time: {numba_time:.4f} seconds")

if numba_time < plain_time:
    speedup_factor = plain_time / numba_time
    print(f"\nNumba provided a speedup of approximately {speedup_factor:.2f} times!")
else:
    print("\nNumba did not provide a speedup in this instance, or the times were similar.")


Results are approximately equal: True
Plain Python Time: 0.8812 seconds
Numba Jitted Time: 0.0218 seconds

Numba provided a speedup of approximately 40.41 times!


### Explanation of Numba Results

As you should observe in the comparison, the Numba-jitted version of `sum_of_sqrts` is significantly faster than the plain Python version. Here's why:

1.  **Just-In-Time Compilation:** The `@njit` decorator tells Numba to compile the Python function into highly optimized machine code when it's first called (the 'warmup' call). This machine code executes much faster than Python bytecode interpreted by the CPython interpreter.
2.  **No Python Interpreter Overhead:** Once compiled, the Numba function runs almost entirely in machine code, avoiding the overhead of the Python interpreter, which is typically a bottleneck for CPU-bound tasks in Python.
3.  **Type Specialization:** Numba infers the types of the variables and operations within the function and generates specialized code for those types, leading to highly efficient execution.
4.  **`math.sqrt` Efficiency:** `math.sqrt` is already a fast C-implemented function. Numba allows the loop structure around this function to be compiled and executed at native speed, further maximizing efficiency.

This example clearly demonstrates Numba's power in accelerating numerical Python code, transforming CPU-bound loops into performant native code.

In [ ]:
def sum_of_sqrts_plain_python(numbers):
    total = 0.0
    for num in numbers:
        total += math.sqrt(num)
    return total

# Warmup run (already done in previous cells, but for self-containment)
_ = sum_of_sqrts_plain_python(data_numba)

start_time_plain = time.time()
plain_result = sum_of_sqrts_plain_python(data_numba)
end_time_plain = time.time()
plain_time = end_time_plain - start_time_plain

print(f"Plain Python Result: {plain_result:.4f}")
print(f"Plain Python Time: {plain_time:.4f} seconds")

Plain Python Result: 21081852648.7170
Plain Python Time: 0.9012 seconds


In [ ]:
@njit
def sum_of_sqrts_numba(numbers):
    total = 0.0
    for num in numbers:
        total += math.sqrt(num)
    return total

# Numba functions have a 'compilation' overhead on the first call.
# We run a warmup call to ensure the timing reflects the compiled version.
_ = sum_of_sqrts_numba(data_numba[:100]) # Use a small slice for warmup

start_time_numba = time.time()
numba_result = sum_of_sqrts_numba(data_numba)
end_time_numba = time.time()
numba_time = end_time_numba - start_time_numba

print(f"Numba Jitted Result: {numba_result:.4f}")
print(f"Numba Jitted Time: {numba_time:.4f} seconds")

Numba Jitted Result: 21081852648.7170
Numba Jitted Time: 0.0220 seconds


In [ ]:
print(f"\nResults are approximately equal: {abs(plain_result - numba_result) < 1e-9}")
print(f"Plain Python Time: {plain_time:.4f} seconds")
print(f"Numba Jitted Time: {numba_time:.4f} seconds")

if numba_time < plain_time:
    speedup_factor = plain_time / numba_time
    print(f"\nNumba provided a speedup of approximately {speedup_factor:.2f} times!")
else:
    print("\nNumba did not provide a speedup in this instance, or the times were similar.")


Results are approximately equal: True
Plain Python Time: 0.9012 seconds
Numba Jitted Time: 0.0220 seconds

Numba provided a speedup of approximately 40.88 times!
